In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns 
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))


df_train = df = pd.read_csv('../input/train.csv',parse_dates=["pickup_datetime","dropoff_datetime"])

df_test = df = pd.read_csv('../input/test.csv',parse_dates=["pickup_datetime"])

In [ ]:
df_train.head()

In [ ]:
df_test.head()

**Lets see if the id columns from both the dataframes are common . **

In [ ]:
train_ids = set(df_train['id'].tolist())
test_ids = set(df_test['id'].tolist())

In [ ]:
len(train_ids.difference(test_ids))

**Looks like the id column has no mapping whatsoever , so we can go ahead and delete that column **

In [ ]:
combine = [df_train,df_test]

for dataset in combine : 
    dataset.drop("id",axis=1,inplace=True)

**Now lets look for the shortest trips in our training dataset . **

In [ ]:
df_train.nsmallest(10,'trip_duration')['trip_duration']

In [ ]:
short_trips_mask = "trip_duration<=1"
len(df_train.query(short_trips_mask)['trip_duration'])

**Looks like 33 of the trips lasted for a second .Thats long , aint it ? **

***Now lets look at the longest trips . *****

In [ ]:
df_train.nlargest(10,'trip_duration')['trip_duration']

In [ ]:
(df_train.nlargest(10,'trip_duration')['trip_duration']/60)/1440

**Woah...Looks like the first four trips(above) have lasted for days (40 days,26 days and so forth)** **Whereas the ones after that have been for a little less than a day . **

**Now lets try to calculate the distances covered in those super long trips that lasted for days.I mean the first four longest trips **

In [ ]:
 from math import sin, cos, sqrt, atan2, radians

In [ ]:
def ride_distance(location_points) : 
    
    from math import sin, cos, sqrt, atan2, radians
    
    R = 6373.0 #(Approximate.Radius of the earth in KM)

    lat1 = radians(location_points[0])
    lon1 = radians(location_points[1])
    lat2 = radians(location_points[2])
    lon2 = radians(location_points[3])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    return format(distance,".2f") #In KM

In [ ]:
df_train.nlargest(4,'trip_duration')[['pickup_latitude','pickup_longitude','dropoff_latitude','dropoff_longitude']].apply(ride_distance,axis=1)

***Hmmm...somethings certainly not right , either with the driver or with the passengers . A 19.91 KM trip lasting for 40 days ?...lol ...these are obvious outliers and do not give us any insights on predicting any attribute whatsoever . But lets investigate further***

In [ ]:
df_train.nlargest(4,'trip_duration')

**Now lets look for areas with the exact same pickup latitudes,longitudes and dropoff latitudes and longitudes****

In [ ]:
same_latlong_mask = "(pickup_latitude == dropoff_latitude) & (dropoff_longitude==pickup_longitude)"

len(df_train.query(same_latlong_mask))

***Alright , so we have 5897 trips which have the exact lame pickup latitudes/longitudes and dropoff latitudes and longitudes . This implies these should be short and quick . lets check out their trip durations***

In [ ]:
df_train.query(same_latlong_mask).nlargest(10,'trip_duration')['trip_duration']

**well well...this tells us the below : **
    
    - There certainly are factors we do not know of. 
    
    - Since trips within the same area have lasted for almost a day .
    
    - The first two longest trips from the above result data (86352 & 85333) have lasted for almost 23 hours and the ones following those two have lasted for like an hour or two . 
    
    

**lets try to grab the Day of week from the datetime columns and look into the weekday , weekend aspects of it . **

In [ ]:
df_train['pickup_day']  = df_train['pickup_datetime'].dt.dayofweek
df_train['dropoff_day']  = df_train['dropoff_datetime'].dt.dayofweek

In [ ]:
df_train.head()

In [ ]:
df_train['trip_distance'] = df_train[['pickup_latitude','pickup_longitude','dropoff_latitude','dropoff_longitude']].apply(ride_distance,axis=1)

In [ ]:
df_train.head()

In [ ]:
df_train['store_and_fwd_flag'] = df_train['store_and_fwd_flag'].map({"N":0,"Y":1})

In [ ]:
df_nearby_trips = df_train.query(same_latlong_mask)

In [ ]:
df_train['trip_distance'] = df_train['trip_distance'].astype("float")
df_train.head()

In [ ]:
vendor1_mask = "vendor_id == 1 "
vendor2_mask = "vendor_id == 2 "

In [ ]:
df_train.groupby("vendor_id").mean()